In [1]:
import os
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Flatten, Dense, Dropout
from keras import optimizers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
batch_size = 32

classes = ['apple', 'banana', 'cherry', 'grape', 'melon', 'orange', 'strawberry']
nb_classes = len(classes)

img_rows = 100
img_cols = 100
channels = 3

train_data_dir = './data/train'
validation_data_dir = './data/validation'

epochs = 50

result_dir = './results'

In [3]:
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

In [4]:
input_tensor = Input(shape=(img_rows, img_cols, 3))
vgg19 = VGG19(include_top=False, weights='imagenet', input_tensor=input_tensor)

80134144/80134624 [============================>.] - ETA: 0s

In [5]:
vgg19.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [7]:
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg19.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

In [8]:
model = Model(inputs=vgg19.input, outputs=top_model(vgg19.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0         
__________

In [10]:
model.layers[:17]

In [11]:
# 最後のconv層の直前までの層をfreeze
for layer in model.layers[:17]:
    layer.trainable = False

In [12]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,# シアー変換
    zoom_range=0.2, #  ランダムに　ズーム
    rotation_range=20, # ランダムな回転回転角度
    horizontal_flip=True, # 水平方向に反転
    vertical_flip=True # 垂直方向に反転
)

In [14]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [15]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_rows, img_cols),
    color_mode='rgb',
    classes=classes,
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True)

Found 4565 images belonging to 7 classes.
Found 697 images belonging to 7 classes.


In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size
)

Epoch 1/50
142/142 [==============================] - 1242s - loss: 1.6152 - acc: 0.4311 - val_loss: 1.5447 - val_acc: 0.4375
Epoch 2/50
142/142 [==============================] - 1218s - loss: 1.2866 - acc: 0.5573 - val_loss: 1.2568 - val_acc: 0.5639
Epoch 3/50
142/142 [==============================] - 1287s - loss: 1.1050 - acc: 0.6340 - val_loss: 1.1370 - val_acc: 0.5940
Epoch 4/50
142/142 [==============================] - 1268s - loss: 0.9942 - acc: 0.6793 - val_loss: 1.0458 - val_acc: 0.6045
Epoch 5/50
142/142 [==============================] - 1273s - loss: 0.9009 - acc: 0.7058 - val_loss: 0.9923 - val_acc: 0.6150
Epoch 6/50
142/142 [==============================] - 1174s - loss: 0.8338 - acc: 0.7202 - val_loss: 0.9549 - val_acc: 0.6466
Epoch 7/50
142/142 [==============================] - 1235s - loss: 0.7977 - acc: 0.7365 - val_loss: 0.9338 - val_acc: 0.6406
Epoch 8/50
142/142 [==============================] - 1259s - loss: 0.7646 - acc: 0.7419 - val_loss: 0.8300 - val_acc:

In [ ]:
# plot results
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(range(epochs), loss, marker='.', label='loss')
plt.plot(range(epochs), val_loss, marker='.', label='val_loss')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
# plot results
loss = history.history['acc']
val_loss = history.history['val_acc']

plt.plot(range(epochs), loss, marker='.', label='acc')
plt.plot(range(epochs), val_loss, marker='.', label='val_acc')
plt.legend(loc='best')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('acc')
plt.show()

In [ ]:
import pickle

with open('./results/history-vgg19_1.pkl', mode='wb') as f:
    pickle.dump(history.history, f)

In [ ]:
model.save('./results/model-vgg19_1.h5')